# 双向循环神经网络
:label:`sec_bi_rnn`

到目前为止，我们处理的序列学习任务示例一直是语言模型，
其中我们的目标是根据序列中的所有先前标记来预测下一个标记。
在这种情况下，我们只希望基于左侧上下文进行条件判断，
因此标准RNN的单向链接似乎是合适的。
然而，在许多其他序列学习任务中，
在每个时间步上基于左侧和右侧上下文进行条件判断是完全合理的。
例如，词性标注。
为什么在评估与给定单词相关的词性时不考虑两个方向的上下文呢？

另一个常见的任务——通常作为实际任务之前的预训练练习——是
在文本文档中随机遮蔽一些标记，然后训练
一个序列模型来预测缺失标记的值。
请注意，根据空白后的内容，
缺失标记的可能值会显著变化：

* 我是`___`。
* 我是`___`饿了。
* 我是`___`饿了，我可以吃掉半头猪。

在第一句中，“快乐”似乎是一个可能的选择。
在第二句中，“不”和“非常”看起来是合理的，
但“不”似乎与第三句不兼容。


幸运的是，一种简单的方法可以将任何单向RNN
转换为双向RNN :cite:`Schuster.Paliwal.1997`。
我们只需实现两个以相反方向链接在一起的单向RNN层
并作用于相同的输入 (:numref:`fig_birnn`)。
对于第一个RNN层，
第一个输入是$\mathbf{x}_1$
最后一个输入是$\mathbf{x}_T$，
但对于第二个RNN层，
第一个输入是$\mathbf{x}_T$
最后一个输入是$\mathbf{x}_1$。
为了生成这个双向RNN层的输出，
我们只需将两个底层单向RNN层的相应输出连接起来即可。


![双向RNN的架构。](../img/birnn.svg)
:label:`fig_birnn`


形式上，对于任何时间步$t$，
我们考虑一个小批量输入$\mathbf{X}_t \in \mathbb{R}^{n \times d}$ 
（样本数量$=n$；每个样本中的输入数量$=d$） 
并设隐藏层激活函数为$\phi$。
在双向架构中，
此时间步的前向和后向隐藏状态分别为$\overrightarrow{\mathbf{H}}_t  \in \mathbb{R}^{n \times h}$ 
和$\overleftarrow{\mathbf{H}}_t  \in \mathbb{R}^{n \times h}$，
其中$h$是隐藏单元的数量。
前向和后向隐藏状态更新如下：


$$
\begin{aligned}
\overrightarrow{\mathbf{H}}_t &= \phi(\mathbf{X}_t \mathbf{W}_{\textrm{xh}}^{(f)} + \overrightarrow{\mathbf{H}}_{t-1} \mathbf{W}_{\textrm{hh}}^{(f)}  + \mathbf{b}_\textrm{h}^{(f)}),\\
\overleftarrow{\mathbf{H}}_t &= \phi(\mathbf{X}_t \mathbf{W}_{\textrm{xh}}^{(b)} + \overleftarrow{\mathbf{H}}_{t+1} \mathbf{W}_{\textrm{hh}}^{(b)}  + \mathbf{b}_\textrm{h}^{(b)}),
\end{aligned}
$$

其中权重$\mathbf{W}_{\textrm{xh}}^{(f)} \in \mathbb{R}^{d \times h}, \mathbf{W}_{\textrm{hh}}^{(f)} \in \mathbb{R}^{h \times h}, \mathbf{W}_{\textrm{xh}}^{(b)} \in \mathbb{R}^{d \times h}, \textrm{ 和 } \mathbf{W}_{\textrm{hh}}^{(b)} \in \mathbb{R}^{h \times h}$，以及偏置$\mathbf{b}_\textrm{h}^{(f)} \in \mathbb{R}^{1 \times h}$ 和 $\mathbf{b}_\textrm{h}^{(b)} \in \mathbb{R}^{1 \times h}$ 都是模型参数。

接下来，我们将前向和后向隐藏状态
$\overrightarrow{\mathbf{H}}_t$ 和 $\overleftarrow{\mathbf{H}}_t$
连接起来，得到用于馈送到输出层的隐藏状态$\mathbf{H}_t \in \mathbb{R}^{n \times 2h}$。
在具有多个隐藏层的深度双向RNN中，
这种信息被作为*输入*传递给下一个双向层。
最后，输出层计算输出
$\mathbf{O}_t \in \mathbb{R}^{n \times q}$（输出数量$=q$）：

$$\mathbf{O}_t = \mathbf{H}_t \mathbf{W}_{\textrm{hq}} + \mathbf{b}_\textrm{q}.$$

这里，权重矩阵$\mathbf{W}_{\textrm{hq}} \in \mathbb{R}^{2h \times q}$ 
和偏置$\mathbf{b}_\textrm{q} \in \mathbb{R}^{1 \times q}$ 
是输出层的模型参数。
虽然从技术上讲，两个方向可以有不同的隐藏单元数，
但在实践中很少做出这样的设计选择。
我们现在展示一个简单的双向RNN实现。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## 从零开始实现

要从零开始实现双向RNN，我们可以
包含两个单向的`RNNScratch`实例
并具有独立的学习参数。

In [2]:
class BiRNNScratch(d2l.Module):
    def __init__(self, num_inputs, num_hiddens, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.f_rnn = d2l.RNNScratch(num_inputs, num_hiddens, sigma)
        self.b_rnn = d2l.RNNScratch(num_inputs, num_hiddens, sigma)
        self.num_hiddens *= 2  # The output dimension will be doubled

前向和后向RNN的状态是分别更新的，而这两个RNN的输出被连接在一起。

In [3]:
@d2l.add_to_class(BiRNNScratch)
def forward(self, inputs, Hs=None):
    f_H, b_H = Hs if Hs is not None else (None, None)
    f_outputs, f_H = self.f_rnn(inputs, f_H)
    b_outputs, b_H = self.b_rnn(reversed(inputs), b_H)
    outputs = [torch.cat((f, b), -1) for f, b in zip(
        f_outputs, reversed(b_outputs))]
    return outputs, (f_H, b_H)

## 简洁实现

使用高级API，
我们可以更简洁地实现双向RNN。
这里我们以GRU模型为例。

In [4]:
class BiGRU(d2l.RNN):
    def __init__(self, num_inputs, num_hiddens):
        d2l.Module.__init__(self)
        self.save_hyperparameters()
        self.rnn = nn.GRU(num_inputs, num_hiddens, bidirectional=True)
        self.num_hiddens *= 2

## 摘要

在双向RNN中，每个时间步的隐藏状态同时由当前时间步之前和之后的数据决定。双向RNN主要用于序列编码和给定双向上下文的观测估计。由于长梯度链的存在，双向RNN的训练成本非常高。

## 练习

1. 如果不同方向使用不同数量的隐藏单元，$\mathbf{H}_t$ 的形状将如何变化？
1. 设计一个具有多个隐藏层的双向RNN。
1. 一词多义在自然语言中很常见。例如，“bank”这个词在“i went to the bank to deposit cash”和“i went to the bank to sit down”两个上下文中具有不同的含义。我们如何设计一个神经网络模型，使得给定一个上下文序列和一个词时，能够返回该词在正确上下文中的向量表示？处理一词多义时首选哪种类型的神经架构？

[讨论](https://discuss.d2l.ai/t/1059)